In [2]:
!pip install pyspark==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 1.7 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 3.5 MB/s eta 0:00:0000:01
  DEPRECATION: pyspark is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for pyspark ... done

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

23/09/23 11:17:57 WARN Utils: Your hostname, Estudos resolves to a loopback address: 127.0.1.1; using 192.168.0.9 instead (on interface enp3s0)
23/09/23 11:17:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/23 11:17:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
df = spark.read.json("../../../datalake/twitter_datascience/")

In [10]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{71, 56, 2023-09...|{[{2023-09-21T08:...|{1234567890abcdef}|  2023-09-21|
|[{37, 21, 2023-09...|{[{2023-09-21T09:...|              null|  2023-09-21|
|[{72, 49, 2023-09...|{[{2023-09-20T06:...|{1234567890abcdef}|  2023-09-20|
|[{2, 10, 2023-09-...|{[{2023-09-20T14:...|              null|  2023-09-20|
|[{58, 69, 2023-09...|{[{2023-09-19T16:...|              null|  2023-09-19|
+--------------------+--------------------+------------------+------------+



In [11]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [12]:
from pyspark.sql import functions as f

In [13]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [14]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{71, 56, 2023-09-...|
|{21, 34, 2023-09-...|
|{93, 76, 2023-09-...|
|{6, 24, 2023-09-2...|
|{94, 12, 2023-09-...|
|{100, 21, 2023-09...|
|{70, 40, 2023-09-...|
|{60, 75, 2023-09-...|
|{60, 40, 2023-09-...|
|{58, 65, 2023-09-...|
|{37, 21, 2023-09-...|
|{32, 55, 2023-09-...|
|{91, 68, 2023-09-...|
|{52, 26, 2023-09-...|
|{41, 52, 2023-09-...|
|{32, 39, 2023-09-...|
|{67, 49, 2023-09-...|
|{81, 37, 2023-09-...|
|{53, 37, 2023-09-...|
|{31, 16, 2023-09-...|
+--------------------+
only showing top 20 rows



In [17]:
df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [18]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [19]:
tweet_df.show()

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       71|             56|2023-09-21T07:10:...| 92|        71|         33|         88|           60|Tweet fictício ge...|
|       21|             34|2023-09-21T03:17:...| 20|         1|         35|         50|           79|Outro tweet fictí...|
|       93|             76|2023-09-21T12:56:...| 84|        41|         89|         64|           43|Este é um tweet f...|
|        6|             24|2023-09-21T03:50:...|  0|        66|         45|         46|            9|Outro tweet fictí...|
|       94|             12|2023-09-21T17:05:...| 68|        57|         27|         40|           32|Um terceiro tweet...|
|      100|     

In [20]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [27]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [30]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [31]:
user_df.show()

+--------------------+---+-------+--------+
|          created_at| id|   name|username|
+--------------------+---+-------+--------+
|2023-09-21T08:16:...| 77| User 1|   user1|
|2023-09-21T03:06:...| 34| User 2|   user2|
|2023-09-21T03:11:...| 26| User 3|   user3|
|2023-09-21T17:05:...| 89| User 4|   user4|
|2023-09-21T20:37:...| 83| User 5|   user5|
|2023-09-21T15:14:...| 40| User 6|   user6|
|2023-09-21T22:35:...| 97| User 7|   user7|
|2023-09-21T19:12:...| 37| User 8|   user8|
|2023-09-21T21:06:...| 14| User 9|   user9|
|2023-09-21T02:34:...| 22|User 10|  user10|
|2023-09-21T09:20:...| 45| User 1|   user1|
|2023-09-21T14:14:...| 94| User 2|   user2|
|2023-09-21T22:33:...| 90| User 3|   user3|
|2023-09-21T05:42:...| 25| User 4|   user4|
|2023-09-21T12:20:...| 95| User 5|   user5|
|2023-09-21T07:49:...|  6| User 6|   user6|
|2023-09-21T12:00:...| 23| User 7|   user7|
|2023-09-21T00:05:...| 44| User 8|   user8|
|2023-09-21T17:40:...| 96| User 9|   user9|
|2023-09-21T11:55:...| 48|User 1

In [32]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')